In [2]:
!pip install transformers datasets accelerate evaluate scikit-learn wandb huggingface_hub
!pip install seqeval

In [3]:
import os
import json
import numpy as np
import torch
from torch import nn
from transformers import (
    AutoTokenizer, AutoModelForTokenClassification, 
    TrainingArguments, Trainer, DataCollatorForTokenClassification,
    EarlyStoppingCallback, TrainerCallback
)
from datasets import load_dataset, DatasetDict
import evaluate
from sklearn.metrics import precision_recall_fscore_support, classification_report
from huggingface_hub import login, create_repo, HfApi
import wandb
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Configuration
MODEL_NAME = "IRIIS-RESEARCH/RoBERTa_Nepali_125M"
# DATASET_NAME = "DipeshChaudhary/error-type-nepali-ged-dataset"
DATASET_NAME1 = "DipeshChaudhary/nepali-gector-style-token-level-tag-for-ged"
ERROR_TYPE_MODEL_HUB_ID = "DipeshChaudhary/nepali-gec-error-type-classifier"

In [5]:
dataset1 = load_dataset(DATASET_NAME1)


Resolving data files:   0%|          | 0/46 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/45 [00:00<?, ?it/s]

In [1]:
# HF_TOKEN = ""

In [7]:
# Login to Hugging Face
login(token=HF_TOKEN, add_to_git_credential=True)

Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [8]:
# Download vocabulary
from huggingface_hub import hf_hub_download
vocab_path = hf_hub_download(
    repo_id="DipeshChaudhary/nepali-gector-style-token-level-tag-for-ged",
    filename="gec_vocabulary.json",
    repo_type="dataset"
)


In [9]:
class GECVocabulary:
    def __init__(self, vocab_path):
        with open(vocab_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            self.tag_to_id = data["tag_to_id"]
            self.id_to_tag = {int(k): v for k, v in data["id_to_tag"].items()}
    
    def get_tag_name(self, tag_id):
        return self.id_to_tag.get(tag_id, "$UNKNOWN")
    
    def get_id(self, tag_name):
        return self.tag_to_id.get(tag_name, 9)
    
    def __len__(self):
        return len(self.tag_to_id)

vocabulary = GECVocabulary(vocab_path)
print(f"✅ Loaded vocabulary: {len(vocabulary)} tags")

✅ Loaded vocabulary: 10 tags


In [10]:
# Hardware setup
NUM_CPUS = 26
BATCH_SIZE = 1024 
os.environ["TOKENIZERS_PARALLELISM"] = "false"

print(f"🚀 Using {NUM_CPUS} CPUs")
print(f"📦 Batch size: {BATCH_SIZE}")
print(f"🤗 Error Type Model Hub: {ERROR_TYPE_MODEL_HUB_ID}")



🚀 Using 26 CPUs
📦 Batch size: 1024
🤗 Error Type Model Hub: DipeshChaudhary/nepali-gec-error-type-classifier


### ===========================================
### ERROR TYPE CLASSIFIER PREPARATION
### ============================================


In [11]:
dataset1['train']['labels'][0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100]

In [12]:
print("\n" + "="*60)
print("STAGE 2: ERROR TYPE CLASSIFIER - FIXED VERSION")
print("="*60)

# First, let's fix the dataset by mapping labels to the correct range
def remap_error_labels(example):
    """Remap original labels (1-7) to (0-6) and filter out others"""
    remapped_labels = []
    for label in example['labels']:
        if label == -100:  # Keep padding
            remapped_labels.append(-100)
        elif 1 <= label <= 7:  # Map DELETE(1)-MERGE_PREV(7) to 0-6
            remapped_labels.append(label - 1)
        else:  # Filter out KEEP(0), SPLIT(8), UNKNOWN(9)
            remapped_labels.append(-100)
    return {'remapped_labels': remapped_labels}

print("🔄 Remapping error labels to 0-6 range...")
error_dataset_remapped = dataset1.map(
    remap_error_labels,
    batched=False,
    num_proc=NUM_CPUS,
    desc="Remapping error labels"
)



STAGE 2: ERROR TYPE CLASSIFIER - FIXED VERSION
🔄 Remapping error labels to 0-6 range...


In [13]:
error_dataset_remapped['train']['labels'][0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100]

In [14]:
error_dataset_remapped['train']['remapped_labels'][0]

[-100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 1,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100]

In [15]:

# # Remove the old labels column and rename
error_dataset_final = error_dataset_remapped.remove_columns(['labels']).rename_column('remapped_labels', 'labels')


In [16]:
error_dataset_final['train']['labels'][0]

[-100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 1,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100]

In [17]:
# Verification
print("🔍 Verifying label ranges after remapping...")
def verify_label_ranges(dataset_split):
    all_labels = []
    for example in dataset_split.select(range(50000)):
        for label in example['labels']:
            if label != -100:
                all_labels.append(label)
    
    unique_labels = sorted(set(all_labels))
    print(f"Remapped unique labels: {unique_labels}")
    print(f"Label range: {min(unique_labels)} to {max(unique_labels)}")
    
    # Should only contain 0-6
    valid_labels = set(range(7))
    invalid_labels = [label for label in unique_labels if label not in valid_labels and label != -100]
    if invalid_labels:
        print(f"❌ STILL INVALID LABELS: {invalid_labels}")
    else:
        print("✅ All labels are in valid range 0-6")

verify_label_ranges(error_dataset_final['train'])

🔍 Verifying label ranges after remapping...


Remapped unique labels: [0, 1, 2, 3, 4, 5, 6]
Label range: 0 to 6
✅ All labels are in valid range 0-6


In [18]:
# compute metrics for error type classification
def compute_error_type_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    
    # We only care about error positions (labels != 0)
    true_predictions = []
    true_labels = []
    
    for prediction, label in zip(predictions, labels):
        for p, l in zip(prediction, label):
            if l != -100 and l != 0:  # Only error positions (ignore KEEP and padding)
                true_predictions.append(p)
                true_labels.append(l)
    
    if len(true_predictions) == 0:
        return {'accuracy': 0, 'macro_f1': 0}
    
    accuracy = (np.array(true_predictions) == np.array(true_labels)).mean()
    
    # Per-class metrics for error types
    error_type_names = ["DELETE", "REPLACE", "APPEND", "SWAP_NEXT", "SWAP_PREV", "MERGE_NEXT", "MERGE_PREV"]
    error_type_ids = [0,1, 2, 3, 4, 5, 6]
    
    # Filter to only include present classes
    present_classes = [cls for cls in error_type_ids if cls in true_labels]
    if present_classes:
        report = classification_report(
            true_labels, true_predictions, 
            labels=present_classes,
            output_dict=True,
            zero_division=0
        )
        macro_f1 = report['macro avg']['f1-score']
    else:
        macro_f1 = 0
    
    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'num_error_tokens': len(true_predictions)
    }


In [19]:
def get_error_type_weights(dataset_split, vocabulary):
    """Get error type weights - simple and reliable approach"""
    print("📊 Calculating error type weights...")
    
    # Using a reasonable sample size
    sample_size = min(10000, len(dataset_split))
    sample = dataset_split.select(range(sample_size))
    
    error_counts = Counter()
    
    # Count error tokens directly from labels
    for i in range(len(sample)):
        example = sample[i]
        for label in example['labels']:
            if label not in [-100] and 0 <= label <= 6:  # Error tokens we care about
                error_counts[label] += 1
    
    total_errors = sum(error_counts.values())
    
    print(f"\nError distribution from {sample_size:,} examples:")
    for tag_id in sorted(error_counts.keys()):
        tag_name = vocabulary.get_tag_name(tag_id+1)
        count = error_counts[tag_id]
        percentage = (count / total_errors) * 100 if total_errors > 0 else 0
        print(f"  {tag_name} ({tag_id}): {count:,} ({percentage:.2f}%)")
    
    # Calculate balanced weights
    class_weights = {}
    for tag_id in sorted(error_counts.keys()):  # All error types we want
        count = error_counts.get(tag_id, 1)
        if total_errors > 0:
            weight = total_errors / (7 * count)  # 7 error types
            class_weights[tag_id] = min(weight, 5.0)  # Reasonable cap
        else:
            class_weights[tag_id] = 1.0
    
    print("\nFinal class weights:")
    for tag_id in sorted(class_weights.keys()):
        tag_name = vocabulary.get_tag_name(tag_id+1)
        print(f"  {tag_name}: {class_weights[tag_id]:.2f}")
    
    return class_weights, error_counts

# Use this - it will work
error_weights, error_counts = get_error_type_weights(error_dataset_final['train'], vocabulary)

📊 Calculating error type weights...

Error distribution from 10,000 examples:
  $DELETE (0): 7 (0.09%)
  $REPLACE (1): 2,847 (34.69%)
  $APPEND (2): 2,196 (26.76%)
  $SWAP_NEXT (3): 1,577 (19.22%)
  $SWAP_PREV (4): 1,577 (19.22%)
  $MERGE_NEXT (5): 1 (0.01%)
  $MERGE_PREV (6): 1 (0.01%)

Final class weights:
  $DELETE: 5.00
  $REPLACE: 0.41
  $APPEND: 0.53
  $SWAP_NEXT: 0.74
  $SWAP_PREV: 0.74
  $MERGE_NEXT: 5.00
  $MERGE_PREV: 5.00


In [20]:
error_counts

Counter({1: 2847, 2: 2196, 3: 1577, 4: 1577, 0: 7, 5: 1, 6: 1})

In [21]:
# Custom loss for error type classification
class WeightedErrorLoss(nn.Module):
    def __init__(self, class_weights):
        super().__init__()
        self.class_weights = torch.tensor(list(class_weights.values()), dtype=torch.float32)
        self.ce_loss = nn.CrossEntropyLoss(weight=self.class_weights, reduction='none')
    
    def forward(self, logits, labels):
        # Move weights to correct device
        self.class_weights = self.class_weights.to(logits.device)
        
        # Calculate loss
        loss = self.ce_loss(logits.view(-1, logits.size(-1)), labels.view(-1))
        
        # Mask: only compute loss for error tokens (labels != 0 and != -100)
        mask = (labels.view(-1) != -100) & (labels.view(-1) != 0)
        masked_loss = loss * mask.float()
        
        # Average only over error tokens
        num_error_tokens = mask.sum()
        if num_error_tokens > 0:
            return masked_loss.sum() / num_error_tokens
        else:
            return masked_loss.sum()  # Returns 0 if no errors

## trying aother things - end

In [22]:
print("📥 Loading model for error type classification...")
error_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# We'll use only the error types that exist: 1-7 (DELETE to MERGE_PREV)
error_id2label = {id-1: name for id, name in vocabulary.id_to_tag.items() if id in [1,2,3,4,5,6,7]}
error_label2id = {name: id for id, name in error_id2label.items()}

print(f"Error types: {error_id2label}")

error_model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(error_id2label),
    id2label=error_id2label,
    label2id=error_label2id,
)

# Apply custom loss
error_model.loss_fct = WeightedErrorLoss(error_weights)

# Data collator
error_data_collator = DataCollatorForTokenClassification(
    tokenizer=error_tokenizer,
    padding=True,
    max_length=128,
    pad_to_multiple_of=8
)


📥 Loading model for error type classification...


Error types: {0: '$DELETE', 1: '$REPLACE', 2: '$APPEND', 3: '$SWAP_NEXT', 4: '$SWAP_PREV', 5: '$MERGE_NEXT', 6: '$MERGE_PREV'}


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at IRIIS-RESEARCH/RoBERTa_Nepali_125M and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
error_id2label

{0: '$DELETE',
 1: '$REPLACE',
 2: '$APPEND',
 3: '$SWAP_NEXT',
 4: '$SWAP_PREV',
 5: '$MERGE_NEXT',
 6: '$MERGE_PREV'}

In [24]:
error_label2id

{'$DELETE': 0,
 '$REPLACE': 1,
 '$APPEND': 2,
 '$SWAP_NEXT': 3,
 '$SWAP_PREV': 4,
 '$MERGE_NEXT': 5,
 '$MERGE_PREV': 6}

In [25]:
error_training_args = TrainingArguments(
    output_dir="./error-type-model",
    overwrite_output_dir=True,
    num_train_epochs=3,  # More epochs for finer classification
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=1,
    learning_rate=2e-6,  # Lower learning rate for fine-grained task
    warmup_steps=500,
    weight_decay=0.01,
    max_grad_norm=1.0,
    
    # Evaluation & Saving
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=3,
    
    # Hub Uploading
    push_to_hub=True,
    hub_model_id=ERROR_TYPE_MODEL_HUB_ID,
    hub_strategy="every_save",
    hub_token=HF_TOKEN,
    
    # Optimization
    dataloader_num_workers=NUM_CPUS,
    dataloader_pin_memory=True,
    fp16=True,
    tf32=True,
    
    # Metrics
    load_best_model_at_end=True,
    metric_for_best_model="eval_macro_f1",
    greater_is_better=True,
    report_to= None,
)


In [27]:
filtered_dataset = error_dataset_final.map(
    lambda batch: {
        "keep_mask": [not v for v in batch["is_correct"]]
    },
    batched=True,
    num_proc=NUM_CPUS
).filter(
    lambda example: example["keep_mask"],
    num_proc=NUM_CPUS
).remove_columns(["keep_mask"])

Map (num_proc=26):   0%|          | 0/13008711 [00:00<?, ? examples/s]

Map (num_proc=26):   0%|          | 0/2439231 [00:00<?, ? examples/s]

Map (num_proc=26):   0%|          | 0/813050 [00:00<?, ? examples/s]

Filter (num_proc=26):   0%|          | 0/13008711 [00:00<?, ? examples/s]

Filter (num_proc=26):   0%|          | 0/2439231 [00:00<?, ? examples/s]

Filter (num_proc=26):   0%|          | 0/813050 [00:00<?, ? examples/s]

In [28]:
filtered_dataset['test']['labels'][999]

[-100,
 -100,
 1,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100]

In [29]:
error_trainer = Trainer(
    model=error_model,
    args=error_training_args,
    train_dataset=filtered_dataset["train"],
    eval_dataset=filtered_dataset["validation"],
    data_collator=error_data_collator,
    tokenizer=error_tokenizer,
    compute_metrics=compute_error_type_metrics,
)


In [30]:
os.environ["WANDB_DISABLED"] = "true"

In [31]:
print("🚀 Training error type classifier...")
error_train_result = error_trainer.train()
error_trainer.save_model()
error_trainer.push_to_hub()

print("✅ Error type model training completed!")
print(f"Final error type metrics: {error_train_result.metrics}")


🚀 Training error type classifier...


wandb: Currently logged in as: 9804234394d (9804234394d-Everest Engineering College) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,Macro F1,Num Error Tokens
1000,0.594400,0.426629,0.829424,0.539446,1934543
2000,0.368200,0.282098,0.891865,0.588158,1934543
3000,0.297200,0.235052,0.911127,0.602718,1934543
4000,0.261800,0.207750,0.922088,0.610916,1934543
5000,0.237900,0.190043,0.929187,0.616294,1934543
6000,0.221700,0.178010,0.933868,0.619842,1934543
7000,0.206600,0.167541,0.937823,0.622804,1934543
8000,0.197700,0.160428,0.940477,0.624811,1934543
9000,0.192000,0.154629,0.942523,0.626339,1934543
10000,0.183600,0.150248,0.944283,0.627649,1934543


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

No files have been modified since last commit. Skipping to prevent empty commit.


✅ Error type model training completed!
Final error type metrics: {'train_runtime': 7470.6076, 'train_samples_per_second': 2611.979, 'train_steps_per_second': 2.551, 'total_flos': 1.2747329669015378e+18, 'train_loss': 0.24652013110073548, 'epoch': 3.0}
